In [1]:
import torch 
import numpy 
dataset_name='ogbl-ddi'
import matplotlib
import matplotlib.pyplot as plt
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
import torch_geometric 
import myutils
import models
dataset=PygLinkPropPredDataset(name=dataset_name)
from torch_geometric.utils import negative_sampling

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Resources used for inspiration on code:
graph exploration from :
https://medium.com/mlearning-ai/ultimate-guide-to-graph-neural-networks-1-cora-dataset-37338c04fe6f

# Exploring our data

In [ ]:
data=dataset[0]
print(f'the {dataset_name} has {len(dataset)} graph')
print(f'number of nodes:{data.num_nodes}')
print(f'number of edges {data.num_edges}')
print(f'number of features {data.num_node_features}')
print(f'is data-graph directed? :{data.is_directed()}')
print(f'data has self-loops? : {data.has_self_loops()}')
print(f'data has isolated nodes? : {data.has_isolated_nodes()}')
print('the graph has average node degree of {:.2f}'.format(data.num_edges/data.num_nodes))

Data edges are given as two arrays 
array[0][i] holds the edge to array [1][i]
we look only on one array and infer the second
 

Helper functions

nx gets edges a stules


In [ ]:
#returns a tensor with the indices of neighbors of the node index
def get_neighbors(edge_index,node_index):
    edge_index=edge_index
    return edge_index[:,numpy.where(edge_index[0]==node_index)[0]][1]

import networkx as nx
def visualize_nx(edges_list):
    unique_list=numpy.unique(edges_list)
    print(f'the graph has {unique_list.shape} nodes')
    myGraph=nx.Graph()
    myGraph.add_nodes_from(unique_list)
    
    myGraph.add_edges_from(list(zip(edges_list[0],edges_list[1])))
    plt.figure()
    nx.draw_networkx(myGraph,with_labels=True)

    plt.show()




In [ ]:
edges2d_t=data.edge_index
node_example_t=edges2d_t[:,numpy.where(edges2d_t[0]==4)[0]]
node_example_n=node_example_t.numpy()[:,:5]
myGraph=nx.Graph()
myGraph.add_nodes_from(numpy.unique(node_example_n))
myGraph.add_edges_from(list(zip(node_example_n[0],node_example_n[1])))



nx.draw_networkx(myGraph,with_labels=False)

In [ ]:

node_example_t=edges2d_t[:,numpy.where(edges2d_t[0]==4)[0]].numpy()
node_example_t=node_example_t[:,:5]
node_example_t


In [ ]:
%%script echo skipping
myGraph=nx.Graph()
myGraph.add_nodes_from(data.edge_index[0])
myGraph.add_edges_from(list(zip(data.edge_index[0],data.edge_index[1])))

In [ ]:
import pandas 
def draw_degree_histogram(data):
    myGraph=nx.to_networkx_graph(list(zip(data[0].numpy(),data[1].numpy())))
    degrees=[val for (node,val) in myGraph.degree()]
    
    plt.hist(degrees,bins=range(0,max(degrees)+1))
    ax=plt.gca()
    plt.xlabel("# of interactions per drug (degree)")
    ax.set_ylim([0,30])
    plt.show()
    print(pandas.DataFrame(degrees).describe().transpose().round(3))



In [ ]:
draw_degree_histogram(data.edge_index)

In [ ]:
from pylab import rcParams
def draw_most_important(data):
    myGraph=nx.to_networkx_graph(list(zip(data[0].numpy(),data[1].numpy())))
    
    color_lookup={node:degree for node,degree in sorted(myGraph.degree())}
    print(color_lookup)

In [ ]:
myGraph=nx.to_networkx_graph(list(zip(data.edge_index[0].numpy(),data.edge_index[1].numpy())))
node_degree_sequence=numpy.array(object= sorted({(n,d) for (n,d) in myGraph.degree()},reverse=True,key=lambda x:x[1]))
node_degree_sequence[:10]

In [ ]:
low,high=node_degree_sequence[:,1].min(),node_degree_sequence[:,1].max()
print(f'low degree:{low}, high degree:{high}')

In [ ]:
node1=5
degreenode1=myGraph.degree(node1)
print(f'node {node1} has degree {degreenode1}')

nx.draw(
    G=myGraph,
    nodelist=[node1],
    node_color='red',
    with_labels=False,
)
plt.show()

In [ ]:
pos=nx.spring_layout(myGraph)
cent=nx.degree_centrality(myGraph)
node_size=list(map(lambda x:x*50,cent.values()))
cent_array=numpy.array(list(cent.values()))
threshold=sorted(cent_array,reverse=True)[10]
print(f'threshold:{threshold}')
cent_bin=numpy.where(cent_array>threshold,1,0.1)
plt.figure(figsize=(15,12))
nodes=nx.draw_networkx_nodes(
    G=myGraph,
    pos=pos,
    node_size=node_size,
    cmap=plt.cm.plasma,
    nodelist=list(cent.keys()),
    alpha=cent_bin,
    node_color=cent_bin

)
edges=nx.draw_networkx_edges(
    G=myGraph,
    pos=pos,
    width=0.03, alpha=0.2
)

## Get the sparse matrix version of the data.

We use ToSparseTensor to get a Tensor object with key adj_t

In [4]:
dataset_sparse=PygLinkPropPredDataset(name='ogbl-ddi', transform=torch_geometric.transforms.ToSparseTensor())
device='cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
data_sparse=dataset_sparse[0]
adj_t=data_sparse.adj_t.to(device)
type(adj_t)

c:\Users\pchrk\OneDrive\Desktop\pms cs\NLP\NLP\nlp_venv\lib\site-packages\torch_geometric\utils\sparse.py:176: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ..\aten\src\ATen\SparseCsrTensorImpl.cpp:56.)
  return adj.to_sparse_csr()


torch.Tensor

In [ ]:
split_edge=dataset.get_edge_split()
split_edge.items()

# Initial Features and Benchmark
## Topological similarity features
1. common neighbors
2. Jaccard's coefficient
3. Adamic/adar
4. Preferential attachment


In [ ]:
data=dataset[0]
Graph_total=nx.to_networkx_graph(list(zip(data.edge_index[0].numpy(), data.edge_index[1].numpy())))
pagerank_f=nx.pagerank(Graph_total, alpha=0.85)
clustering_coef_f=nx.clustering(Graph_total)
betweenness_f=nx.betweenness_centrality(Graph_total)
# adamic_adar_f=nx.adamic_adar_index(Graph_total)
# betweenness_f=nx.betweenness_centrality(Graph_total)




In [ ]:
common_neighbor_centrality_f=nx.common_neighbor_centrality(Graph_total)
save_to_txt("common_neighbor_centrality",common_neighbor_centrality_f)

In [ ]:

# feutures_emb=torch.ones(data.num_nodes,10, dtype=torch.float64).to(device)
# for _ in range(data.num_nodes):
#     features_emb[_][0]=pagerank_f[_]
#     features_emb[_][1]=clustering_coef_f[_]
#     features_emb[_][2]=betweenness_f[_]


In [ ]:
embedding_1s=torch.ones(data.num_nodes,dtype=torch.float64)
embedding_1s.to(device)

In [5]:

dataset=PygLinkPropPredDataset(name=dataset_name)
data=dataset[0]
split_edge=dataset.get_edge_split()
train_data=split_edge['train']
pagerank_f=myutils.get_dict_from_file("pagerank.txt")
clustering_coef_f=myutils.get_dict_from_file("clustering_coef.txt")
betweenness_f=myutils.get_dict_from_file("betweeness_centrality.txt")



In [4]:
from random import randint
m=len(train_data['edge'])
X_train=torch.zeros(2*len(train_data['edge']),3, dtype=torch.float64)
y_train=torch.zeros(2*len(train_data['edge']),1,dtype=torch.float64)
for x,edge in enumerate(train_data['edge']):
    X_train[x][0]=pagerank_f[edge[0].item()]+pagerank_f[edge[1].item()]
    X_train[x][1]=clustering_coef_f[edge[0].item()]+clustering_coef_f[edge[1].item()]
    X_train[x][2]=betweenness_f[edge[0].item()]+betweenness_f[edge[1].item()]

    y_train[x]=1

    random_node1=randint(0,data.num_nodes-1)
    random_node2=randint(0,data.num_nodes-1)
    X_train[m+x][0]=pagerank_f[random_node1]+pagerank_f[random_node2]
    X_train[m+x][1]=clustering_coef_f[random_node1]+clustering_coef_f[random_node2]
    X_train[m+x][2]=betweenness_f[random_node1]+betweenness_f[random_node2]

    y_train[m+x]=0
    #print("filling x at {0} %".format(x/m*100))

X_train.shape


    

torch.Size([2135822, 3])

In [11]:
test_data=split_edge['test']
X_test=torch.zeros(len(test_data['edge'])+len(test_data['edge_neg']),3, dtype=torch.float64)
y_test=torch.zeros(len(test_data['edge'])+len(test_data['edge_neg']),1,dtype=torch.float64)
for x,node in enumerate(test_data['edge']):
    X_test[x][0]=pagerank_f[node[0].item()]+pagerank_f[node[1].item()]
    X_test[x][1]=clustering_coef_f[node[0].item()]+clustering_coef_f[node[1].item()]
    X_test[x][2]=betweenness_f[node[0].item()]+betweenness_f[node[1].item()]
    y_test[x]=0

for x, node in enumerate(test_data['edge_neg']):
    X_test[len(test_data['edge'])+x][0]=pagerank_f[node[0].item()]+pagerank_f[node[1].item()]
    X_test[len(test_data['edge'])+x][1]=clustering_coef_f[node[0].item()]+clustering_coef_f[node[1].item()]
    X_test[len(test_data['edge'])+x][2]=betweenness_f[node[0].item()]+betweenness_f[node[1].item()]
    y_test[len(test_data['edge'])+x]=1
    

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
n_epochs=1000
loss_fn=torch.nn.BCELoss()
my_clf=LogisticRegressionModel(input_dim=3, output_dim=1)
X_train.to(device)
y_train.to(device)
optimizer=torch.optim.SGD(my_clf.parameters(), lr=0.01)
for epoch in range(n_epochs):
    y_pred=my_clf(X_train)
    loss=loss_fn(y_pred,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')


In [12]:
test_data=split_edge['test']
test_data['edge']

tensor([[2198, 1172],
        [1205,  719],
        [1818, 2866],
        ...,
        [ 326, 1109],
        [ 911, 1250],
        [4127, 2480]])

In [6]:
evaluator=Evaluator(name='ogbl-ddi')
evaluator.expected_input_format

"==== Expected input format of Evaluator for ogbl-ddi\n{'y_pred_pos': y_pred_pos, 'y_pred_neg': y_pred_neg}\n- y_pred_pos: numpy ndarray or torch tensor of shape (num_edges, ). Torch tensor on GPU is recommended for efficiency.\n- y_pred_neg: numpy ndarray or torch tensor of shape (num_edges, ). Torch tensor on GPU is recommended for efficiency.\ny_pred_pos is the predicted scores for positive edges.\ny_pred_neg is the predicted scores for negative edges.\nNote: As the evaluation metric is ranking-based, the predicted scores need to be different for different edges."

# Basic SAGE model
We used the example at [the official ogb repo](https://github.com/snap-stanford/ogb/blob/master/examples/linkproppred/ddi/gnn.py) to define our SAGE class
We used moduleList because we do not know in advane how many convolutions we will nedd to use




In [19]:
dataset = PygLinkPropPredDataset(name='ogbl-ddi')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data = dataset[0]
split_edge = dataset.get_edge_split()

The SAGE classes work with adjency data for efficiency so we are going to use the adjacency matrix representation

In [20]:
dataset_sparse=PygLinkPropPredDataset(name='ogbl-ddi', transform=torch_geometric.transforms.ToSparseTensor())

In [7]:
data=dataset_sparse[0]
#because of the transform we know have a sparse tensor
adj_t=data.adj_t.to(device)
split_edges=dataset_sparse.get_edge_split()
#example of adj_tensor of node no2
adj_t[2].flatten()

tensor(indices=tensor([[   9,   13, 2254, 2415]]),
       values=tensor([1., 1., 1., 1.]),
       size=(4267,), nnz=4, layout=torch.sparse_coo)

In [25]:
# returns node features with dimensions 256
from torch_geometric.nn import SAGEConv
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, number_layers,out_channels):
        super(SAGE, self).__init__()

        self.convs=torch.nn.ModuleList()
        self.convs.append(
            SAGEConv(in_channels, hidden_channels)
        )
        for x in range(number_layers-2):
            self.convs.append(
                SAGEConv(hidden_channels, hidden_channels)
            )
        self.convs.append(SAGEConv(hidden_channels, out_channels))
                          
    def forward(self,x,adjacency_t):
        # ommit the last convolutional layer
        for conv in self.convs[:-1]:
            x=conv(x,adjacency_t)
            x=torch.nn.functional.relu(x)
    #at least one layer is present
        x=self.convs[-1](x,adjacency_t)
        return x
    

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
    

In [68]:
class SimpleLinkPredictor(torch.nn.Module):
    def __init__(self,in_channels,hidden_channels,out_channels):
        super(SimpleLinkPredictor, self).__init__()

        self.lin1=torch.nn.Linear(in_channels, hidden_channels)
        self.lin2=torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x_i, x_j):
        x=x_i*x_j
        x=self.lin1(x)
        x=torch.nn.functional.relu(x)
        x=self.lin2(x)
        return torch.nn.functional.sigmoid(x).squeeze()


    def reset_parameters(self):
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()


In [76]:
class DotLinkPredictor(torch.nn.Module):
    def __init__(self):
        super(DotLinkPredictor, self).__init__()
        
    def forward(self, x_i, x_j):
        out = (x_i*x_j).sum(-1)
        #print(out)
        return torch.sigmoid(out)
    

    def reset_parameters(self):
        pass

# Simple Link Predictor with SAGE model


In [29]:
hidden_dim=256
model=SAGE(1,hidden_channels=hidden_dim, number_layers=3,out_channels=hidden_dim).to(device)
dotLinkPredictor=DotLinkPredictor().to(device)

## set nodes to evaluation mode
model.eval()
#set the link predictor to evaluation mode
dotLinkPredictor.eval()

DotLinkPredictor()

In [11]:

initial_embeddings=torch.ones(size=(data.num_nodes,1)).to(device)

In [ ]:
torch.random.manual_seed(1955)
number_of_edges=split_edges['train']['edge'].size(0)
#returns a tensor of size 0 to 1067911 (the number of edges in the trainin portion of the dataset)
rand_index=torch.randperm(number_of_edges)[:10]
rand_edges_t=split_edges['train']['edge'][rand_index]
rand_edges_t


In [14]:
def create_train_batch(all_pos_train_edges,perm,edge_index):
    pos_edges=all_pos_train_edges[perm].t().to(device)

    neg_edges=negative_sampling(edge_index, num_neg_samples=pos_edges.shape[1]).to(device)
    training_edges=torch.cat([pos_edges, neg_edges], dim=1)

    pos_labels=torch.ones(pos_edges.shape[1], dtype=torch.float, device=device)
    neg_labels=torch.zeros(neg_edges.shape[1], dtype=torch.float, device=device)

    training_labels=torch.cat([pos_labels, neg_labels], dim=0).to(device)

    return training_edges, training_labels

In [21]:
dataset=PygLinkPropPredDataset(name=dataset_name)
data=dataset[0]
split_edge=dataset.get_edge_split()

In [22]:
from torch.utils.data import DataLoader
for perm in DataLoader(range(32), batch_size=16, shuffle=True):
    batch=create_train_batch(split_edge['train']['edge'], perm, data.edge_index)
    print (batch) 

(tensor([[4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039,
         4039, 4039, 4039, 4039, 1655, 1315, 3185, 4009, 2919, 2822, 4000, 2783,
         2632, 1277, 4248,  329, 2217,  548,  350, 4120],
        [1331,  313,  474,  738,  511, 3978,  405, 2337, 1150, 2336, 3037,  779,
         2521,  122,  785, 2392,  559, 2692,  620, 2806, 1760, 3336, 3259,  805,
         1740,  631, 3803, 1324, 2778,  497, 3356, 2228]]), tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))
(tensor([[4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039, 4039,
         4039, 4039, 4039, 4039, 1532, 2384, 2430, 1316, 3667, 2760, 3882, 4225,
         3888, 2211, 2148,  585,  684, 2041, 2541, 1210],
        [3832,  223,  225,  308, 3646, 2221, 1279, 2235, 2424, 1734, 3667,  476,
          608,  346,  802, 3901, 1188, 4026, 2994, 3315, 3892, 2085, 1346, 2910,
          607, 2003, 2040, 1

In [95]:
# part of the training function
model=SAGE(1,hidden_channels=hidden_dim, number_layers=3,out_channels=hidden_dim).to(device)
torch.manual_seed(1955)
edge_index=data.edge_index.to(device)
dataset_sparse=PygLinkPropPredDataset(name='ogbl-ddi', transform=torch_geometric.transforms.ToSparseTensor())
adj_t=dataset_sparse[0].adj_t.to(device)
dotLinkPredictor=DotLinkPredictor().to(device)
simplePredictor=SimpleLinkPredictor(in_channels=hidden_dim, hidden_channels=hidden_dim, out_channels=1).to(device)
#x are the init embeddings of the nodes
initial_embeddings=torch.ones(size=(data.num_nodes,1)).to(device)
def train(model,predictor,x,adj_t,split_edge,optimizer,batch_size,num_epochs):
    
    model.train()
    
    model.reset_parameters()
    predictor.reset_parameters()

    all_pos_train_edges=split_edge['train']['edge']
    all_pos_train_edges.to(device)  
    for epoch in range(num_epochs):
        total_epoch_loss=0
        for perm in DataLoader(range(all_pos_train_edges.shape[0]), batch_size=batch_size, shuffle=True):
            #set gradient to zero before starting backpropagation
            optimizer.zero_grad()
            batch=create_train_batch(split_edge['train']['edge'], perm, edge_index)
            train_edge, train_labels=batch

            h=model(x, adj_t)
            preds=dotLinkPredictor(h[train_edge[0]], h[train_edge[1]])
            #print(simplePredictor(h[train_edge[0]], h[train_edge[1]]))
            #print(train_labels)
            loss=torch.nn.functional.binary_cross_entropy(preds, train_labels)
            total_epoch_loss+=loss.item()
        
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch}, loss {total_epoch_loss}')
           

In [96]:
optimizer=torch.optim.Adam(list(model.parameters())+list(dotLinkPredictor.parameters()), lr=0.01)
train(model, dotLinkPredictor, initial_embeddings, adj_t, split_edge, optimizer, 256, 10)

KeyboardInterrupt: 

In [ ]:
def 